In [1]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import os
import ast
import random
import pandas as pd
from transformers import TrainingArguments, AutoTokenizer, HfArgumentParser
from utils.my_trainer import CustomTrainer
from utils.utils import my_compute_metrics,seed_everything
from typing import Optional
from dataclasses import dataclass, field
from model.my_model import PPathVLM, WPathVLM
from peft import LoraConfig
from datasets import load_dataset, concatenate_datasets, Dataset
from utils.data_collator import MyDataCollatorForWPathVLM

/bask/projects/p/phwq4930-gbm/Zeyu/asiw9691_conda_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set up tokenizer
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3-8B-Instruct')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.truncation_side = 'left'

new_tokens = ['<|Question|>',  '<|Answer|>', '<Image>']  
num_added_toks = tokenizer.add_tokens(new_tokens)
new_tokens_ids = tokenizer.convert_tokens_to_ids(new_tokens)
print("new_tokens_ids: ", new_tokens_ids)

questions = pd.read_csv('./utils/question_wsi_list.csv', header=None)  
questions = questions[0].tolist()

def formatting_func(examples):
    question = random.choice(questions)
    answer = examples["label"]
    text = f"<|Question|> {question}{tokenizer.eos_token} " + f"<|Answer|> {answer}{tokenizer.eos_token}\n"
    examples["text"] = text
    return examples

dataset = []
eval_dataset = None

dataset = Dataset.load_from_disk('/bask/projects/p/phwq4930-gbm/Zeyu/WSI_Dataset/WVLMdata_test')
dataset = dataset.map(formatting_func, num_proc=20, remove_columns=['label'])
train_dataset = dataset

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


new_tokens_ids:  [128256, 128257, 128258]


In [3]:
model = WPathVLM(False, 
                False, 
                False, 
                'meta-llama/Meta-Llama-3-8B-Instruct', 
                False, 
                True, 
                tokenizer,
                new_tokens_ids[-1])

llm loading ...


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.55it/s]


In [4]:
training_args = TrainingArguments(
    output_dir='./output',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    # gradient_checkpointing=True,
    learning_rate=2e-5,
    # lr_scheduler_type="cosine",
    logging_steps=5,
    num_train_epochs=1,
    max_steps=-1,
    report_to='wandb',
    save_steps=50,
    save_total_limit=5,
    bf16=True,
    warmup_ratio=0.1,
    evaluation_strategy='steps',
    eval_steps=10000,
    logging_first_step=True,
    remove_unused_columns=False,
    label_names=["labels"]
)

peft_config = None
data_collator = MyDataCollatorForWPathVLM(tokenizer)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    max_seq_length=256,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field='text',
    peft_config=peft_config,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=my_compute_metrics,
)

/bask/projects/p/phwq4930-gbm/Zeyu/PathVLM/source/PathLLM/utils/my_trainer.py:352: UserWarning: You passed `remove_unused_columns=False` on a non-packed dataset. This might create some issues with the default collator and yield to errors. If you want to inspect dataset other columns (in this case ['f1', 'f3', 'cor3', 'f2', 'cor2', 'text', 'cor1']), you can subclass `DataCollatorForLanguageModeling` in case you used the default collator and create your own data collator in order to inspect the unused dataset columns.
  warnings.warn(
/bask/projects/p/phwq4930-gbm/Zeyu/asiw9691_conda_env/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, 

In [5]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: betpotti (pathlmm). Use `wandb login --relogin` to force relogin


OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 39.39 GiB of which 69.06 MiB is free. Including non-PyTorch memory, this process has 39.32 GiB memory in use. Of the allocated memory 38.81 GiB is allocated by PyTorch, and 8.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x150a20ad5d80>> (for post_run_cell), with arguments args (<ExecutionResult object at 150bc641c4c0, execution_count=5 error_before_exec=None error_in_exec=CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 39.39 GiB of which 69.06 MiB is free. Including non-PyTorch memory, this process has 39.32 GiB memory in use. Of the allocated memory 38.81 GiB is allocated by PyTorch, and 8.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables) info=<ExecutionInfo object at 150bc641e7d0, raw_cell="trainer.train()" store_history=True silent=False shell_futures=True cell_id=f0402c39-feb5-4a09-875b-a008e4a2a51c> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given